In [6]:
import pandas as pd
import glob
import os

# from google.colab import drive
# drive.mount('/content/drive')
data_dir = "/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/ANPHY-Sleep/EPCTL_all_subj_sleep_labels"
results_dir = "/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/results"

file_list = glob.glob(os.path.join(data_dir, "*.txt"))

print("Files found:", file_list)

Files found: ['/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/ANPHY-Sleep/EPCTL_all_subj_sleep_labels/EPCTL28 copy.txt', '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/ANPHY-Sleep/EPCTL_all_subj_sleep_labels/EPCTL29 copy.txt', '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/ANPHY-Sleep/EPCTL_all_subj_sleep_labels/EPCTL16 copy.txt', '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/ANPHY-Sleep/EPCTL_all_subj_sleep_labels/EPCTL17 copy.txt', '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/ANPHY-Sleep/EPCTL_all_subj_sleep_labels/EPCTL22 copy.txt', '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/ANPHY-Sleep/EPCTL_all_subj_sleep_labels/EPCTL23 copy.txt', '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/ANPHY-Sleep/EPCTL_all_subj_sleep_labels/EPCTL01 copy.txt', '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/ANPHY-Sleep/EPCTL_all_subj_sleep_labels/EPCTL

In [7]:
stage_mapping = {
    'W': 0,
    'N1': 1,
    'N2': 2,
    'N3': 3,
    'R': 4
}

df_list = []

for file_path in file_list:
    df = pd.read_csv(file_path, sep='\t', header=None, names=['Stage', 'Start', 'Duration'])
    df = df[df['Stage'] != 'L']
    df = df[['Stage', 'Start']]

    patient_id = os.path.splitext(os.path.basename(file_path))[0]
    df['Patient'] = patient_id
    df['Patient'] = df['Patient'].str.replace(" copy", "", regex=False)
    df['Stage'] = df['Stage'].map(stage_mapping)
    df_list.append(df)

dataset = pd.concat(df_list, ignore_index=True)

dataset = dataset[['Patient', 'Stage', 'Start']]
print(dataset)

aggregated = dataset.groupby("Patient").agg({
    "Stage": lambda x: list(x),
    "Start": lambda x: list(x)
}).reset_index()

print(aggregated.head())

       Patient  Stage  Start
0      EPCTL28      0   4860
1      EPCTL28      0   4890
2      EPCTL28      0   4920
3      EPCTL28      0   4950
4      EPCTL28      0   4980
...        ...    ...    ...
24188  EPCTL04      0  22350
24189  EPCTL04      0  22380
24190  EPCTL04      0  22410
24191  EPCTL04      0  22440
24192  EPCTL04      0  22470

[24193 rows x 3 columns]
   Patient                                              Stage  \
0  EPCTL01  [0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, ...   
1  EPCTL02  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2  EPCTL03  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, ...   
3  EPCTL04  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4  EPCTL05  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                               Start  
0  [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, ...  
1  [1290, 1320, 1350, 1380, 1410, 1440, 1470, 150...  
2  [2130, 2160, 2190, 2220, 2250, 2280, 2310, 234...  
3  [450, 480, 5

In [8]:
#aggregated.to_csv("combined_sleep_data.csv", index=False)
aggregated.to_csv(os.path.join(results_dir, "combined_sleep_data.csv"), index=False)